In [13]:
%load_ext autoreload
%autoreload 2

import json
from itertools import product
import logging

import core.datacache as datacache
import core.filters as filters

# import pipeliner
# import indexer
# from core.trainingjob import TrainingJob
# import core.indexer as indexer


logging.basicConfig(
    level=logging.INFO,
    format="'%(name)s - %(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ]
)


def generate_combinations(steps):
    methods = []
    for step in steps.values():
        methods.append([(m, step["type"]) for m in step['method']])
    return list(product(*methods))


# TRAINING
training_config = {
    "seed": [42],  # int
    "random_scope": "dataset",  # str - ["dataset", "global", "models"]
    # List[str] - [path1, path2, ...]
    "paths": [("data/test_data", [1,2,3]), "data/_Raisin/Raisin_Tavernier_830_GFratio", "data/_RefSet/ALPINE_C_424_Murguzur_RMSE1.16"],
    # List[(Splitter, Dict)]  - [splitter, params] -> to create dataset indexes trees
    "indexation": [],
    "pre_indexation": {
        "step_1": {
            "type": "filter",
            "method": [("crop", filters.Crop, {"start":100, "end":500}), ("crop", filters.Crop, {"start":0, "end":1000}), None],
        },
        "step_2": {
            "type": "filter",
            "method": [("resample", filters.Uniform_FT_Resample, {"resample_size": 800}), None],
        }
    },
    "post_indexation": {
        "step_1": {
            "type": "augmentation",
            "method": [],  # List[List[(TransformerMixin, Dict)]]
        },
        "step_2": {
            "type": "preprocessing",
            "method": [],  # List[List[(TransformerMixin, Dict)]]
        },
    },
    "models": [
        # List[(Estimator, Dict)]  - [estimator, params] -> to create sklearn pipeline
        [],
    ],
}

# def


def train(config):
    for dataset_path in config["paths"]:
        logging.info(f"Processing dataset {dataset_path}")
        # 1. Load data
        dataset_uid, dataset_name, data = datacache.register_dataset(dataset_path)
        
        # 2 .Filter data
        pre_indexation_steps = generate_combinations(config["pre_indexation"])
        logging.info(pre_indexation_steps)
        
        # 3. Indexation
        # if not pre_indexation_steps:
        #     pre_indexation_steps = [None]

        # for pre_indexation_step in pre_indexation_steps:
        #     indexations = indexer.get(dataset, config["indexation"], pre_indexation_step)
        #     post_indexation_steps = generate_combinations(config["post_indexation"])
        #     runs = generate_combinations(indexations, post_indexation_steps, config["models"])
        #     for run in runs:
        #         scheduler.add(run)


train(training_config)
print("end")

'root - 2023-06-02 17:16:16,135 [INFO] Processing dataset ('data/_Raisin/Raisin_Tavernier_830_GFratio', [1, 2, 3])
'root - 2023-06-02 17:16:16,136 [INFO] Registering dataset: ('data/_Raisin/Raisin_Tavernier_830_GFratio', [1, 2, 3])
'root - 2023-06-02 17:16:16,138 [INFO] Loading file: data\_Raisin\Raisin_Tavernier_830_GFratio\Xcal.csv
'root - 2023-06-02 17:16:16,142 [INFO] Header inference: False
'root - 2023-06-02 17:16:16,143 [INFO] Delimiter inference: ;
'root - 2023-06-02 17:16:16,325 [INFO] Data shape: (664, 125)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'root - 2023-06-02 17:16:16,330 [INFO] Data shape after dropna(all) cols: (664, 125)
'root - 2023-06-02 17:16:16,335 [INFO] Data shape after dropna(all) rows: (664, 125)
'root - 2023-06-02 17:16:16,337 [INFO] Rows to remove after filtering: [432, 434, 428, 438]
'root - 2023-06-02 17:16:16,338 [INFO] data_sample:
[[0.2384144 0.2335801]
 [0.248429  0.2475333]]
'root - 2023-06-02 17:16:16,341 [INFO] Loading file: data\_Raisin\Raisin_Tavernier_830_GFratio\Xval.csv
'root - 2023-06-02 17:16:16,344 [INFO] Header inference: False
'root - 2023-06-02 17:16:16,345 [INFO] Delimiter inference: ;
'root - 2023-06-02 17:16:16,410 [INFO] Data shape: (166, 125)
'root - 2023-06-02 17:16:16,414 [INFO] Data shape after dropna(all) cols: (166, 125)
'root - 2023-06-02 17:16:16,418 [INFO] Data shape after dropna(all) rows: (166, 125)
'root - 2023-06-02 17:16:16,420 [INFO] data_sample:
[[0.2509339 0.2475901]
 [0.2625283 0.2588682]]


AssertionError: Cannot initialize y columns ([1, 2, 3]). More than one X_train file found for Raisin_Tavernier_830_GFratio.

In [ ]:
%load_ext autoreload
%autoreload 2
import core.datacache as datacache
import numpy as np
import logging

logging.basicConfig(
    level=logging.INFO,
    format="'%(name)s - %(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("debug.log"),
        # logging.StreamHandler()
    ]
)

# data = datacache.load_csv("data/_RefSet/ALPINE_C_424_Murguzur_RMSE1.16/XCal.csv.gz")
# cache_hash, dataset_name, cache = datacache.register_dataset("data/test_data")
# print(cache_hash, dataset_name)
# for k, v in cache.items():
#     if isinstance(v, np.ndarray):
#         print(k, v.shape)

cache_hash, dataset_name, cache = datacache.register_dataset("data/test_data_2")
print(cache_hash, dataset_name)
for k, v in cache.items():
    if isinstance(v, np.ndarray):
        print(k, v.shape)


# data.isna().any(axis=0)

# hex(hash(str(data)))



In [ ]:
import numpy as np


def remove_nan_rows(X, y):
    nan_indexes_X = np.isnan(X).any(axis=1)
    nan_indexes_y = np.isnan(y).any(axis=0)

    removed_indexes_X = np.where(nan_indexes_X)[0]
    removed_indexes_y = np.where(nan_indexes_y)[0]
    

    matching_indexes_X = np.delete(np.arange(len(X)), removed_indexes_X)
    matching_indexes_y = np.delete(np.arange(len(y)), removed_indexes_y)

    X_matched = X[matching_indexes_X]
    y_matched = y[matching_indexes_y]

    return X_matched, y_matched


# Example usage
# Assuming you have X_train and y_train as the original arrays

# Generate example arrays with NaN values
X_train = np.array([[1, 2, np.nan], [4, 5, 6], [np.nan, 8, 9]])
y_train = np.array([10, 11, np.nan])

# Remove rows with NaN values and get matching arrays
X_matched, y_matched = remove_nan_rows(X_train, y_train)

print("X_matched:\n", X_matched)
print("y_matched:\n", y_matched)